In [1]:
#%load_ext tensorboard

#import datetime
import numpy as np
import scipy.io as sio
#import matplotlib as plt

import os

#added
import h5py

#import tensorflow as tf


import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#tf.compat.v1.disable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True)) #check if gpu is used
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
    
with tf.Session() as sess:
  devices = sess.list_devices()
devices

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0

Default GPU Device: /device:GPU:0


[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 974782997815470631),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 31592420480, 7121197767071902605)]

### Load the data

In [3]:
vols = h5py.File("/scratch/hterborg/brute_force/Thesis/zero_3_runs_15_train_1_test_dataset.mat",'r')
                 
                 
                 #15train_onetest_dataset.mat",'r')

In [4]:
vols

list(vols.keys())

['test_x', 'test_y', 'testsubject', 'train_x', 'train_y', 'trainsubjects']

In [5]:
X_train, y_train, X_test, y_test = vols['train_x'], vols['train_y'], vols['test_x'], vols['test_y']
n_classes = 2

In [6]:
np.size(X_train)

2745103850

In [7]:
X_train=np.transpose(X_train)
y_train=np.transpose(y_train)



In [8]:
X_test=np.transpose(X_test)
y_test=np.transpose(y_test)

In [9]:
print(np.shape(X_train),np.shape(y_train))

(4630, 79, 95, 79) (4630, 1)


In [10]:
#X_train = X_train[:1000,:,:,:]
#y_train = y_train[:1000]

In [11]:
trainsubjects = np.transpose(vols['trainsubjects'])
print(trainsubjects)

[[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 16.]]


In [12]:
y_train

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=uint8)

In [13]:
print(np.shape(X_train),np.shape(y_train))

(4630, 79, 95, 79) (4630, 1)


### z-scoring to normalize the data

In [14]:
x_mean = np.mean(X_train)
x_std = np.std(X_train)

In [15]:
X_tr_centered = (X_train - x_mean)/x_std
X_ts_centered = (X_test - x_mean)/x_std


### 1D array for labels

In [16]:
y_train = y_train.flatten()
y_test = y_test.flatten()

#### Checking out the dimension

In [17]:
X_tr_centered.shape,  X_ts_centered.shape, y_train.shape

((4630, 79, 95, 79), (316, 79, 95, 79), (4630,))

In [18]:
np.std(X_tr_centered)

0.9999993

#### Checking out the image 

### Batch generator: to generate mini-batches for training 

In [20]:
def batch_generator(X, y, batch_size=50, 
                    shuffle=True, random_seed=None):
    
    idx = np.arange(y.shape[0])
    
    if shuffle:
        rng = np.random.RandomState(random_seed)
        rng.shuffle(idx)
        X = X[idx]
        y = y[idx]
    
    for i in range(0, X.shape[0], batch_size):
        #print(i)
        yield (X[i:i+batch_size, :], y[i:i+batch_size])

### 3D-CNN class

In [21]:
class Conv3dNN(object):
    def __init__(self, n_classes=2, batchsize=50,
                 epochs=10, learning_rate=1e-4, 
                 dropout_rate=0.5,
                 shuffle=True, random_seed=None):
        np.random.seed(random_seed)
        self.batchsize = batchsize
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate
        self.shuffle = shuffle
        self.n_classes = n_classes
                
        g = tf.Graph()
        with g.as_default():
            ## set random-seed:
            tf.set_random_seed(random_seed)
            
            ## build the network:
            self.build()

            ## initializer
            self.init_op = \
                tf.global_variables_initializer()

            ## saver
            self.saver = tf.train.Saver()
            
        ## create a session
        self.sess = tf.Session(graph=g)
                
    def build(self):
        
        ## Placeholders for X and y:
        tf_x = tf.placeholder(tf.float32, 
                              shape=[None, 79, 95, 79],
                              name='tf_x')
        tf_y = tf.placeholder(tf.int32, 
                              shape=[None],
                              name='tf_y')
        is_train = tf.placeholder(tf.bool, 
                              shape=(),
                              name='is_train')
        
        ## reshape x to 5D tensor:
        ## [batchsize, x, y, z, 1]
        tf_x_vol = tf.reshape(tf_x, shape=[-1, 79, 95, 79, 1],
                             name='input_x_3d_volumes')

        ## One-hot encoding:
        tf_y_onehot = tf.one_hot(indices=tf_y, depth=2,
                              dtype=tf.float32,
                              name='input_y_onehot')

        ## 1st layer: Conv_1
        h1 = tf.layers.conv3d(tf_x_vol, 
                              filters=8, 
                              kernel_size=(7, 7, 7), 
                              strides=(1, 1, 1),
                              padding='valid',
                              activation=tf.nn.relu)
        ## MaxPooling
        h1_pool = tf.layers.max_pooling3d(h1, 
                              pool_size=(2, 2, 2), 
                              strides=(2, 2, 2))
        
        ## 2nd layer: Conv_2
        h2 = tf.layers.conv3d(h1_pool, 
                              filters=16, 
                              kernel_size=(5, 5, 5), 
                              strides=(1,1,1),
                              padding='valid',
                              activation=tf.nn.relu)
        ## MaxPooling 
        h2_pool = tf.layers.max_pooling3d(h2, 
                              pool_size=(2, 2, 2), 
                              strides=(2, 2, 2))

        ## 3rd layer: Conv_3
        h3 = tf.layers.conv3d(h2_pool, 
                              filters=32, 
                              kernel_size=(3, 3, 3), 
                              strides=(1,1,1),
                              padding='valid',
                              activation=tf.nn.relu)
        ## MaxPooling 
        h3_pool = tf.layers.max_pooling3d(h3, 
                              pool_size=(2, 2, 2), 
                              strides=(2, 2, 2))
        
        ## 4th layer: Fully Connected
        input_shape = h3_pool.get_shape().as_list()
        n_input_units = np.prod(input_shape[1:])
        h3_pool_flat = tf.reshape(h3_pool, 
                              shape=[-1, n_input_units])
        
        h4 = tf.layers.dense(h3_pool_flat, 128, 
                              activation=tf.nn.relu)

        ## Dropout
        h4_drop = tf.layers.dropout(h4, 
                              rate=self.dropout_rate,
                              training=is_train)
        
        ## 5th layer: Fully Connected (linear activation)
        h5 = tf.layers.dense(h4_drop, self.n_classes, 
                              activation=tf.nn.sigmoid)

        ## Prediction
        predictions = {
            'probabilities': tf.nn.softmax(h5, 
                              name='probabilities'),
            'labels': tf.cast(tf.argmax(h5, axis=1), 
                              tf.int32, name='labels')}
        
        ## Loss Function and Optimization
        cross_entropy_loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                logits=h5, labels=tf_y_onehot),
            name='cross_entropy_loss')
        
        ## Optimizer:
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        optimizer = optimizer.minimize(cross_entropy_loss,
                              name='train_op')

        ## Finding accuracy
        correct_predictions = tf.equal(
            predictions['labels'], 
            tf_y, name='correct_preds')
        
        accuracy = tf.reduce_mean(
            tf.cast(correct_predictions, tf.float32),
            name='accuracy')

    def save(self, epoch, path='./CNN3d-tflayers-modeltest/'):
        if not os.path.isdir(path):
            os.makedirs(path)
        print('Saving model in %s' % path)
        self.saver.save(self.sess, 
                        os.path.join(path, 'zero_model_10.ckpt'),
                        global_step=epoch)
        
    def load(self, epoch, path):
        print('Loading model from %s' % path)
        self.saver.restore(self.sess, 
             os.path.join(path, 'zero_model_10.ckpt-%d' % epoch))
        
    def train(self, training_set, 
              validation_set=None,
              initialize=True):
        ## initialize variables
        if initialize:
            self.sess.run(self.init_op)

        self.train_cost_ = []

        for epoch in range(1, self.epochs + 1):      avg_loss = 0.0
            for batch_x,batch_y in batch_generator(np.array(training_set[0]),np.array(training_set[1]), batch_size=self.batchsize,shuffle=self.shuffle):
               
                feed = {'tf_x:0': batch_x, 
                        'tf_y:0': batch_y,
                        'is_train:0': True} ## for dropout
                loss, _ = self.sess.run(
                        ['cross_entropy_loss:0', 'train_op'], 
                        feed_dict=feed)
                avg_loss += loss
                
            print('Epoch %02d: Training Avg. Loss: '
                  '%7.3f' % (epoch, avg_loss), end=' ')
            if validation_set is not None:
                
                X_data_ts = np.array(training_set[0])
                y_data_ts = np.array(training_set[1])
                # test accuracy
                batch_gen_ts = \
                batch_generator(X_data_ts, y_data_ts,
                                 shuffle=False,batch_size=self.batchsize)
                avg_valid_acc = 0.0
                for i, (batch_x,batch_y) in \
                    enumerate(batch_gen_ts):
                    feed = {'tf_x:0': batch_x,
                            'tf_y:0': batch_y,
                            'is_train:0': False} ## for dropout
                    avg_valid_acc = avg_valid_acc + self.sess.run('accuracy:0', feed_dict=feed)
                avg_valid_acc = avg_valid_acc/(i+1)
                
                print('Validation Acc: %7.3f' % avg_valid_acc)
            else:
                print()
                    
    def predict(self, X_test, return_proba = False):
        feed = {'tf_x:0': X_test,
                'is_train:0': False} ## for dropout
        if return_proba:
            return self.sess.run('probabilities:0',
                                 feed_dict=feed)
        else:
            return self.sess.run('labels:0',
                                 feed_dict=feed)

### Create an instance of the CNN3dNN class, train it, and save the trained model

In [22]:
#(len(X_tr_centered)/3)

In [23]:
cnn3d = Conv3dNN( random_seed=123,epochs=10, n_classes=n_classes)

cnn3d.train(training_set=(X_tr_centered, y_train), 
         validation_set=(X_ts_centered, y_test))

cnn3d.save(epoch=10)


/home/hterborg/.local/lib/python3.7/site-packages/tensorflow/python/keras/legacy_tf_layers/convolutional.py:628: UserWarning: `tf.layers.conv3d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv3D` instead.
  warnings.warn('`tf.layers.conv3d` is deprecated and '
/home/hterborg/.local/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/home/hterborg/.local/lib/python3.7/site-packages/tensorflow/python/keras/legacy_tf_layers/pooling.py:464: UserWarning: `tf.layers.max_pooling3d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling3D` instead.
  warnings.warn('`tf.layers.max_pooling3d` is deprecated and '


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



/home/hterborg/.local/lib/python3.7/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/home/hterborg/.local/lib/python3.7/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:268: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  warnings.warn('`tf.layers.dropout` is deprecated and '


Epoch 01: Training Avg. Loss:  58.280 Validation Acc:   0.667
Epoch 02: Training Avg. Loss:  53.499 Validation Acc:   0.679
Epoch 03: Training Avg. Loss:  52.977 Validation Acc:   0.674
Epoch 04: Training Avg. Loss:  53.606 Validation Acc:   0.697
Epoch 05: Training Avg. Loss:  52.923 Validation Acc:   0.690
Epoch 06: Training Avg. Loss:  52.700 Validation Acc:   0.721
Epoch 07: Training Avg. Loss:  52.449 Validation Acc:   0.737
Epoch 08: Training Avg. Loss:  52.224 Validation Acc:   0.740
Epoch 09: Training Avg. Loss:  51.555 Validation Acc:   0.736
Epoch 10: Training Avg. Loss:  51.262 Validation Acc:   0.770
Saving model in ./CNN3d-tflayers-modeltest/


In [24]:

#cnn3d_re.train(training_set=(X_tr_centered, y_train), 
         validation_set=(X_ts_centered, y_test),initialize=False)
#[0:round((len(X_tr_centered)/4))]
#cnn3d.save(epoch=10)

IndentationError: unexpected indent (<ipython-input-24-a99e12e87171>, line 2)

### To load the trained model and to test it using data 

In [29]:
#del cnn3d

cnn3d_re = Conv3dNN(random_seed=123)
cnn3d_re.load(epoch=10, path='./CNN3d-tflayers-modeltest/')

print(cnn3d_re.predict(X_ts_centered[:10,:,:,:]))


Loading model from ./CNN3d-tflayers-modeltest/
INFO:tensorflow:Restoring parameters from ./CNN3d-tflayers-modeltest/zero_model_10.ckpt-10
[1 1 1 1 1 1 1 1 1 1]


### Accuray for all test data

In [ ]:
X_ts_centered2 = X_ts_centered[100:300,:,:,:]
y_test2 =y_test[100:300]

In [31]:
preds = cnn3d_re.predict(X_ts_centered)

print('Test Accuracy: {:.2f}'.format( 100 * np.sum(y_test == preds)/len(y_test)))



Test Accuracy: 77.85


In [32]:
diff = y_test-preds
print('diff: ',diff)

# Correct is 0 
# FP is -1 
# FN is 1
print('Correctly classified: ', np.where(diff == 0)[0])
print('Incorrectly classified: ', np.where(diff != 0)[0])
print('False positives: ', np.where(diff == -1)[0])
print('False negatives: ', np.where(diff == 1)[0])

diff:  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0 -1 -1  0 -1 -1 -1 -1 -1 -1  0 -1  0  0 -1 -1 -1 -1 -1
  0  0  0  0  0  0  0  0 -1  0 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  1  1  0  0  1  1  0  1  0  0  0  0  0  0  0  0  1
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0 -1 -1 -1 -1  0  0
  0  0  0  0  0 -1 -1 -1  0  0 -1 -1 -1  0  0  0  0 -1 -1  0  0  0 -1  0
  0  0  0  0  0  0  0  0  0  1  1  0  0  0  0  0  1  0  0  0  0  0  1  1
  0  0  1  1  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1  0 -1  0
  0 -1  0  0  0 -1 -1  0  0 -1 -1 -1 -1  0  0 -1  0 -1 -1 -1 -1  0  0  0
  0  0  0 -1  0  0  0  0  0  0  0  1  0  0  0  0  0  1  0  1  0  0  0  0
  0  0  0  0  0  0  0  1  0  0  1  0  0  0  1  1  0  0  0  0  0  0  0  0
  0  0  0  0]
Correctly classified:  [  0   

In [33]:
np.size(np.where(diff == -1)) #false positive


48

In [34]:
np.size(np.where(diff == 1)) # false negative

22